In [ ]:
import wave
song = wave.open("song_embedded.wav", mode='rb')
# Convert audio to byte array
frame_bytes = bytearray(list(song.readframes(song.getnframes())))

# Extract the LSB of each byte
extracted = [frame_bytes[i] & 1 for i in range(len(frame_bytes))]
# Convert byte array back to string
string = "".join(chr(int("".join(map(str,extracted[i:i+8])),2)) for i in range(0,len(extracted),8))
# Cut off at the filler characters
decoded = string.split("###")[0]

# Print the extracted text
print("Sucessfully decoded: "+decoded)
song.close()

In [1]:
import wave
# read wave audio file
with wave.open("Ex3_sound1.wav", mode='rb') as wav:
    f_bytes = bytearray(list(wav.readframes(wav.getnframes())))
    

In [2]:
extracted_bytes = [f_bytes[i] & 1 for i in range(len(f_bytes))]
secret_message = "".join(chr(int("".join(map(str,extracted_bytes[i:i+8])),2)) for i in range(0,len(extracted_bytes),8))

In [3]:
decrypted_message = secret_message.split("###")[0]

# Print the extracted text
print("Secret message is : "+decrypted_message)
wav.close()

Secret message is : Mom I will not be home for supper
